In [1]:
import pandas as pd
from datetime import datetime
import pytz

In [18]:
# Load the dataset
data = pd.read_csv("/Users/sarabjotsingh/Downloads/car_prices.csv")  # Replace "your_dataset.csv" with the path to your dataset

# Preprocessing steps
# Remove rows with missing values
data.dropna(inplace=True)

# Remove duplicate rows
data.drop_duplicates(inplace=True)

# Remove rows where 'year' column is less than 2000
data = data[data['year'] >= 2000]

# Remove rows in 'model' column containing numeric data
data = data[~data['model'].str.contains(r'\d', regex=True)]

# Remove rows where 'transmission' column contains 'sedan'
data = data[data['transmission'] != 'sedan']

# Remove specified columns
columns_to_drop = ['vin', 'condition']
data.drop(columns=columns_to_drop, inplace=True)

# Convert 'odometer' column to integer
data['odometer'] = data['odometer'].astype(int)

In [19]:
# Convert 'saledate' column to string type
data['saledate'] = data['saledate'].astype(str)

# Split 'saledate' column into separate 'date' and 'time' columns
data[['date', 'time']] = data['saledate'].str.split(' ', n=1, expand=True)

# Define time zones
pst_tz = pytz.timezone('America/Los_Angeles')  # PST
est_tz = pytz.timezone('America/New_York')     # EST

# Convert 'date' and 'time' columns to datetime format and apply timezone conversions
data['date'] = pd.to_datetime(data['date'])
data['time'] = pd.to_datetime(data['time']).dt.time
data['date'] = data['date'].dt.tz_localize(pst_tz).dt.tz_convert(est_tz)

# Combine 'date' and 'time' columns into a single column
data['converted_saledate'] = data['date'].dt.strftime('%Y-%m-%d') + ' ' + data['time'].astype(str)

# Drop 'date' and 'time' columns
data.drop(columns=['date', 'time'], inplace=True)

/var/folders/h_/kssb81453xb2cjn36fxdy_z40000gn/T/ipykernel_6110/1598720480.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['date'] = pd.to_datetime(data['date'])


DateParseError: Unable to parse datetime string: Tue, at position 0

In [20]:
# Print sample values from the 'date' column
print(data['date'].head())


0    Tue
1    Tue
5    Tue
7    Tue
9    Tue
Name: date, dtype: object


In [ ]:
# Save the updated dataset
data.to_csv("converted_data.csv", index=False)

print("Conversion completed and saved to converted_data.csv")
